# Запросы на отрезках

## Префиксные последовательности

**Определение:** Префиксная последовательность от последовательности $A_n$ и обратимой операции $\circ$ - это

\begin{align}
S_n(\omega) =
\begin{cases}
0, & \mbox{ } n = 0 \\
A_0, & \mbox{ } n = 1 \\
S_{n - 1} \circ A_{n - 1}, & \mbox{ } n > 1
\end{cases}
\end{align}

Важнейшим свойством префиксной последовательности является то, что $A_l \circ A_{l + 1} \circ ... \circ A_r = S_r \circ S_l^{-1}$. Это позволяет выполнять запросы на отрезке массива за $O(1)$

__Замечание:__ Свойством обратимости обладают например сумма, произведение, исключающее или, сложение по модулю и другие групповые операции. Зато им не обладают min/max

Построение префиксной последовательности:

Для того чтобы классический **префиксный массив** поддерживал операцию «⊕» на префиксах, достаточно, чтобы на множестве значений была задана структура **моноида**, то есть:

1. **Ассоциативность**

   $$
     (a\;\⊕\;b)\;\⊕\;c \;=\; a\;\⊕\;(b\;\⊕\;c)
     \quad\forall a,b,c.
   $$

   Это позволяет наращивать префикс поэлементно:

   $$
     P[i] = P[i-1]\;\⊕\;A[i].
   $$

2. **Нейтральный элемент**
   Существует $e$ такое, что

   $$
     e\;\⊕\;a \;=\; a\;\⊕\;e \;=\; a
     \quad\forall a.
   $$

   Тогда $P[0]=e$ и для любого $i$

   $$
     P[i]=\bigoplus_{j=1}^i A[j].
   $$

> На таком префиксном массиве вы можете быстро отвечать на запросы «сумма» (или иная ⊕-агрегация) на префиксе $[1..r]$ просто как $P[r]$.

---

Если же вы хотите на его основе поддерживать **произвольные диапазонные запросы** $[l..r]$, то требуется ещё и **обратимость** (invertibility), то есть для каждого $a$ есть $a^{-1}$ такая, что

$$
  a\;\⊕\;a^{-1} \;=\; a^{-1}\;\⊕\;a \;=\; e.
$$

В этом случае

$$
  \bigoplus_{i=l}^r A[i]
  = P[r]\;\⊕\;(P[l-1])^{-1}.
$$

Итого:

* **Только префиксные запросы** $[1..r]$ → операция должна образовывать **моноид** (ассоциативность + нейтральный элемент).
* **Произвольные диапазонные запросы** $[l..r]$ → дополнительно нужна **обратимость**, то есть структура должна быть **группой**.

In [14]:
from collections.abc import Callable

def build(A: list, f: Callable[[int, int], int]) -> list:
    n = len(A)
    S = [0] * (n + 1)
    S[1] = A[0]

    for i in range(2, n + 1):
        S[i] = f(S[i - 1], A[i - 1])

    return S

A = [1, 4, 12, 13, 1, 56, 17]
f = lambda x, y: x + y
f_reversed = lambda x, y: y - x
S = build(A, f)
print(S)

[0, 1, 5, 17, 30, 31, 87, 104]


Добавление элементов в префиксную последовательность

In [13]:
def append(S: list, a, f: Callable[[int, int], int]):
    S.append(f(S[-1], a))

append(S, 11, f)
print(S)

[0, 1, 5, 17, 30, 31, 87, 104, 115, 126]


Запрос суммы на отрезке от l до r

In [16]:
l = 2
r = 5
print(f_reversed(S[l], S[r]))

26


## Дерево отрезков

**Определение** Дерево отрезков от последовательности $A_n$ и ассоциативной функции $\phi$ - это дерево, листьями которого являются элементы исходного массива, а вершинами - пары из объединения интервалов их детей и значений ассоциативной функции $\phi$. У каждой вершины по два ребенка.

\+ Merge sort tree

Для того чтобы классический дерево отрезков мог поддерживать операцию “⊕” для запросов на отрезке, достаточно, чтобы на множестве значений массива была задана структура **моноида**, то есть:

1. **Ассоциативность**
   Для любых $a,b,c$

   $$
     (a \;\⊕\; b)\;\⊕\; c \;=\; a\;\⊕\;(b\;\⊕\; c).
   $$

2. **Нейтральный элемент**
   Существует элемент $e$ (аналог «0» или «единицы»), такой что

   $$
     e\;\⊕\;a \;=\; a\;\⊕\;e \;=\; a\quad\forall a.
   $$

> Коммутативность вовсе не обязательна (хотя, разумеется, упрощает некоторые реализации), а обратимость нужна только если вы хотите, помимо чистых запросов “на отрезке”, ещё и «отменять» какие-то обновления через обратный оператор.

---

### Дополнение: ленивая (lazy) сегментная куча

Если вы хотите поддерживать **диапазонные обновления** (например, прибавить к каждому элементу в отрезке $[l..r]$ некое значение) вместе с запросами по другому оператору, то и сами операции обновления и их **композиция** (то есть то, как два последовательных обновления объединяются в одно) тоже должны образовывать моноид:

* Обновление ∘ обновление должно быть ассоциативным.
* Существует «ничто-обновление» (identity update), которое при применении ничего не меняет.

Тогда при «отложенной» (lazy) записи вы можете безопасно накапливать и распределять обновления по дереву, гарантируя корректность и эффективность.

---

#### Вывод

> **Дерево отрезков** поддерживает любую операцию, образующую моноид: ассоциативную бинарную операцию с нейтральным элементом на множестве значений.

In [1]:
n = 7
A = [1, 4, 12, 13, 1, 56, 17]
D = [0] * 3 * n
f = lambda x, y: x + y

Добавление элемента в дерево

In [10]:
def update(v: int, lv: int, rv: int, i: int, val: int):
    if i < lv or i >= rv:
        return
    if lv + 1 == rv:
        D[v] = val
        return
    mid = (lv + rv) // 2
    update(v * 2 + 1, lv, mid, i, val)
    update(v * 2 + 2, mid, rv, i, val)
    D[v] = f(D[v * 2 + 1], D[v * 2 + 2])

def build():
    for i in range(n):
        update(0, 0, n, i, A[i])

build()
print(D)

[104, 17, 87, 1, 16, 14, 73, 0, 0, 4, 12, 13, 1, 56, 17, 0, 0, 0, 0, 0, 0]


In [9]:
def query(v: int, lv: int, rv: int, l: int, r: int):
    if l <= lv and rv <= r:
        return D[v]
    if r <= lv or l >= rv:
        return 0
    mid = (lv + rv) // 2
    return f(query(v * 2 + 1, lv, mid, l, r), query(v * 2 + 2, mid, rv, l, r))

query(0, 0, n, 1, 3)

16

## Декартово дерево

## Дерево Фенвика

Дерево Фенвика (Fenwick Tree) умеет эффективно поддерживать две базовые операции:

1. **Точечное обновление**: добавить (или присвоить) в элементе массива некоторое значение.
2. **Префиксные запросы**: вычислить «сумму» (в более общем смысле — накопленную величину) на префиксе массива $1…i$.

Чтобы обобщить понятие «сумма» на произвольную операцию ⊕, нужно, чтобы она удовлетворяла следующим условиям:

1. **Ассоциативность**
   Для любых a, b, c:

   $$
     (a \;\⊕\; b)\;\⊕\; c \;=\; a\;\⊕\;(b\;\⊕\; c).
   $$

   Это позволяет разбивать префикс на части в произвольном порядке без изменения результата.

2. **Наличие нейтрального элемента**
   Существует элемент e (аналог “0” или “1”), такой что для любого a:

   $$
     e\;\⊕\;a \;=\; a\;\⊕\;e \;=\; a.
   $$

3. **Обратимость (инвертируемость)**
   Для каждого a существует обратный элемент a⁻¹, такой что:

   $$
     a\;\⊕\;a^{-1} \;=\; a^{-1}\;\⊕\;a \;=\; e.
   $$

   Это необходимо, если вы хотите применять не только префиксные, но и **диапазонные** запросы вида
   $l..r$
   по формуле

   $$
     \bigoplus_{i=l}^{r} A[i]
     \;=\;
     \Bigl(\bigoplus_{i=1}^{r} A[i]\Bigr)
     \;\⊕\;\Bigl(\bigoplus_{i=1}^{l-1} A[i]\Bigr)^{-1}.
   $$

> **Итого:** дерево Фенвика поддерживает операцию ⊕ тогда и только тогда, когда на множестве значений массивов операция ⊕ задаёт **группу** (ассоциативность + нейтральный элемент + обратимость). Коммутативность при этом не является жёстким требованием для обработки префиксных запросов (обратимость и ассоциативность достаточно), но, естественно, если ⊕ ещё и коммутативна, то вся логика становится ещё более понятной и универсальной.

In [6]:
class FenwikTree:
    def __init__(self, A):
        self.tree = [0] * len(A)
        self.size = 0
        for i in range(len(A)):
            self.add(i, A[i])

    def add(self, i: int, a: int):
        while i < n:
            self.tree[i] += a
            i |= (i + 1)

    def _query(self, R: int):
        s = 0
        i = R
        while i >= 0:
            s += self.tree[i]
            i = (i & (i + 1)) - 1
        return s

    def query(self, L: int, R: int):
        return self._query(R) - self._query(L - 1)


F = FenwikTree(A)
print(F.tree)
print(F.query(1, 3))

[1, 5, 12, 30, 1, 57, 17]
29


## Sparse table

Для классического Sparse Table достаточно, чтобы операция ⊕ была

1. **Ассоциативной**

   $$
     (a\;\⊕\;b)\;\⊕\;c \;=\; a\;\⊕\;(b\;\⊕\;c).
   $$

2. **Идемпотентной**

   $$
     a\;\⊕\;a \;=\; a.
   $$

Идемпотентность гарантирует, что при запросе

```text
ans = ST[k][l] ⊕ ST[k][r-2^k+1]
```

(где интервалы \[l…l+2ᵏ−1] и \[r−2ᵏ+1…r] могут частично накладываться) «двойной» учёт общих элементов не исказит результата.

Если же вам важно поддерживать и пустые интервалы (или унифицировать интерфейс со структурами наподобие сегментного дерева), имеет смысл также потребовать **нейтральный элемент** e, такой что

$$
  e\;\⊕\;a = a\;\⊕\;e = a.
$$

В сумме это делает (M, ⊕) **идемпотентным моноидом** (или, иначе, полулатисом), на котором и строится Sparse Table.

Помимо уже рассмотренных структур (префиксного массива, Sparse Table, Fenwick-дерева и классического сегментного дерева), в задачах с запросами на отрезке применяют ещё ряд интересных и более специализированных решений:

1. **Sqrt-декомпозиция (móe’s Algorithm)**

   * Разбиение массива на блоки размером \~√N.
   * Позволяет отвечать на запросы онлайн за $O(1)$–$O(\sqrt N)$ в зависимости от типа задачи (для сумм/минимумов–максимумов $O(1)$, для более сложных агрегатов иногда $O(\sqrt N)$).
   * Mo’s Algorithm (алгоритм “метод квадратного корня”) позволяет обрабатывать офлайн-запросы типа «считать число различных элементов на отрезке» за $\,O((N+Q)\sqrt N)$.

2. **Дерево отрезков с «beats» (Segment Tree Beats)**

   * Специализированная версия сегдрева для задач, где требуется поддерживать на отрезке операции типа «присвоить $\min(A[i], x)$» или «увеличить до $\max(A[i], x)$», а также суммировать.
   * Сложность примерно $O(\log N)$ амортизированно за запрос.

3. **Слияние деревьев (Merge Sort Tree)**

   * В каждом узле сегментного дерева хранится отсортированный вектор элементов подотрезка.
   * Позволяет отвечать на «сколько элементов ≤ k на отрезке $[l..r]$» за $O(\log^2 N)$, а также находить k-ю порядковую статистику.

4. **Интервальное дерево (Interval Tree)**

   * Динамическая структура, где каждый узел соответствует какому-то отрезку, хранятся интервалы, пересекающие середину.
   * Часто используется для поддержания множества отрезков и запросов «какие интервалы пересекают точку $x$» или «есть ли непересекающиеся отрезки» за $O(\log N + K)$, где $K$ — число ответов.

5. **Декартово дерево (Cartesian Tree) + LCA**

   * Для статичных RMQ-запросов (Range Minimum Query) строят декартово дерево и сводят к задаче на «наименьшего общего предка» (LCA) в этой дереве.
   * Отвечает за $O(1)$ при $O(N)$ препроцессинге.

6. **Wavelet Tree**

   * Позволяет отвечать на запросы «сколько раз на отрезке $[l..r]$ встретилось значение = k», «сколько элементов ≤ k» или «k-я порядковая статистика» за $O(\log C)$, где $C$ — размер алфавита (диапазон значений).

7. **Range Tree (дерево отрезков в многомерном пространстве)**

   * Обобщение сегдрева на 2D/3D: хранит точки (или объекты) в пространстве, позволяет искать, например, “сколько точек попадают в прямоугольник” за $O(\log^d N)$ при $d$ измерениях.

8. **Персистентный сегментный дерево**

   * Каждая версия дерева сохраняет «срез» старого состояния (копируются только изменённые узлы).
   * Поддерживает онлайн-запросы «какова сумма на отрезке в версии $t$» за $O(\log N)$, а память \~$O((N+Q)\log N)$.

9. **Динамические сбалансированные деревья (Treap, RB-дерево, AVL-дерево)**

   * Хранят массив как двоичное дерево поиска по индексу (implicit key Treap).
   * Позволяют делать вставки/удаления/сдвиги/реверс диапазона и получать агрегат (сумму, минимум) за $O(\log N)$.

10. **Fenwick-дерево в несколько измерений (2D/3D BIT)**

    * Обобщение Fenwick-дерева на решётку, позволяет отвечать на сумму по прямоугольнику за $O(\log^d N)$.

---

Каждая из этих структур подходит для своих типов задач:

* **Статические** (массив не меняется): чаще всего Sparse Table, декартово дерево+LCA, Merge Sort Tree, Wavelet Tree.
* **Динамические** (частые обновления): сегментное дерево (и его производные – Beats, персистентное), Fenwick-дерево, implicit Treap.
* **Офлайн-алгоритмы** (заранее известен весь набор запросов): Mo’s Algorithm, CDQ-разделяй-и-властвуй.

Выбор конкретной структуры определяется требованиями по типу операций (точечное обновление, диапазонное присваивание, поиск порядковой статистики, работа с динамическими вставками/удалениями и т.п.) и жёсткими ограничениями по времени и памяти.